In [126]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_fubon_broker_data(dealer_id,dealer_name,sub_dealer_id,sub_dealer_name,csv_type, start_date, end_date):
    """
    爬取富邦證券卷商買賣超資料
    
    Parameters:
    stock_id (str): 股票代碼
    start_date (str): 開始日期 (格式: YYYY-MM-DD)
    end_date (str): 結束日期 (格式: YYYY-MM-DD)
    
    Returns:
    pandas.DataFrame: 包含卷商買賣超資料的DataFrame
    """
    
    # 構建URL
    url = f"https://fubon-ebrokerdj.fbs.com.tw/z/zg/zgb/zgb0.djhtm"
    params = {
        'a': dealer_name,
        'b': sub_dealer_name,
        'c': csv_type,
        'e': start_date,
        'f': end_date
    }
    # 設定headers模擬瀏覽器
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        # 發送請求
        response = requests.get(url, params=params, headers=headers)
        response.encoding = 'big5'  # 設定編碼為big5
        
        # 解析HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # 找到資料表格
        data = []
        for table in soup.find_all('table', {'class': 't0'}):
            if not table:
                print("找不到資料表格")
                return None

            # 解析表格資料
            rows = table.find_all('tr')[2:]  # 跳過標題行
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 4:  # 確保欄位數量正確
                    broker_type=row.find('script', {'language': 'javascript'})
                    if broker_type is not None:
                        script_text = row.find('script', {'language': 'javascript'}).string
                        stocker = script_text.split("'")[3] 
                        #print(broker)
                    else:
                        stocker = cols[0].text.strip()
                    buy = int(cols[1].text.strip().replace(',', ''))
                    sell = int(cols[2].text.strip().replace(',', ''))
                    net = int(cols[3].text.strip().replace(',', ''))

                    data.append({
                        '券商名稱': sub_dealer_id,
                        '個股名稱': stocker,
                        '買進張數': buy,
                        '賣出張數': sell,
                        '買賣超張數': net
                    })
        
        # 轉換成DataFrame
        df = pd.DataFrame(data)

        response.close()
        
        return df
        
    except Exception as e:
        print(f"發生錯誤: {str(e)}")
        response.close()
        return None

# 使用範
'''
if __name__ == "__main__":
    dealer_id = "9800"
    dealer_name = "元大"
    sub_dealer_id = "9800"
    sub_dealer_name = "元大證卷"
    start_date = "2025-1-21"
    end_date = "2025-1-21"
    
    df = scrape_fubon_broker_data(stock_id, start_date, end_date)
    if df is not None:
        print(df)
        # 可以選擇將結果保存到CSV
        df.to_csv(f'F:\\文義\\股票資料\\股票歷史資料\\卷商分點資料\\(start_date)\\broker_data_{dealer_id}_{dealer_name}_{sub_dealer_id}_{sub_dealer_name}_{start_date}.csv', encoding='utf-8-sig', index=False)
'''       

'\nif __name__ == "__main__":\n    dealer_id = "9800"\n    dealer_name = "元大"\n    sub_dealer_id = "9800"\n    sub_dealer_name = "元大證卷"\n    start_date = "2025-1-21"\n    end_date = "2025-1-21"\n    \n    df = scrape_fubon_broker_data(stock_id, start_date, end_date)\n    if df is not None:\n        print(df)\n        # 可以選擇將結果保存到CSV\n        df.to_csv(f\'F:\\文義\\股票資料\\股票歷史資料\\卷商分點資料\\(start_date)\\broker_data_{dealer_id}_{dealer_name}_{sub_dealer_id}_{sub_dealer_name}_{start_date}.csv\', encoding=\'utf-8-sig\', index=False)\n'

In [130]:
import pandas as pd
import os
import time

# 讀取Excel檔案
df = pd.read_csv('/Users/tai-huachiang/Downloads/卷商分點明細.csv', encoding='big5')

# 新列表存放轉換後的資料
converted_data = []

# 迭代每一筆資料
for index, row in df.iterrows():
    dealer_id = row['dealer_id']
    dealer_name = row['dealer_name']
    sub_dealer_id = row['sub_dealer_id']
    sub_dealer_name = row['sub_dealer_name']
    start_date = "2025-1-22"
    end_date = "2025-1-22"
    csv_type='E'
    
    #if not os.path.exists(f'F:\\文義\\股票資料\\股票歷史資料\\卷商分點資料\\{start_date}'):os.makedirs(f'F:\\文義\\股票資料\\股票歷史資料\\卷商分點資料\\{start_date}')
    if not os.path.exists(f'/Users/tai-huachiang/Downloads/卷商分點資料'):os.makedirs(f'/Users/tai-huachiang/Downloads/卷商分點資料')
    if not os.path.exists(f'/Users/tai-huachiang/Downloads/卷商分點資料/{start_date}'):os.makedirs(f'/Users/tai-huachiang/Downloads/卷商分點資料/{start_date}')

    if os.path.exists(f'/Users/tai-huachiang/Downloads/卷商分點資料/{start_date}/broker_data_張數_{dealer_id}_{dealer_name}_{sub_dealer_id}_{sub_dealer_name}_{start_date}.csv'):continue
    df = scrape_fubon_broker_data(dealer_id,dealer_name,sub_dealer_id,sub_dealer_name,csv_type, start_date, end_date)
    if df is not None:
        print('資料get:'+str(dealer_id)+str(dealer_name)+str(sub_dealer_id)+str(sub_dealer_name))
        # 可以選擇將結果保存到CSV
        #df.to_csv(f'F:\\文義\\股票資料\\股票歷史資料\\卷商分點資料\\{start_date}\\broker_data_{dealer_id}_{dealer_name}_{sub_dealer_id}_{sub_dealer_name}_{start_date}.csv', encoding='utf-8-sig', index=False)
        if csv_type=='E':
            df.to_csv(f'/Users/tai-huachiang/Downloads/卷商分點資料/{start_date}/broker_data_張數_{dealer_id}_{dealer_name}_{sub_dealer_id}_{sub_dealer_name}_{start_date}.csv', encoding='utf-8-sig', index=False)
        else:
            df.to_csv(f'/Users/tai-huachiang/Downloads/卷商分點資料/{start_date}/broker_data金額_{dealer_id}_{dealer_name}_{sub_dealer_id}_{sub_dealer_name}_{start_date}.csv', encoding='utf-8-sig', index=False)
    else:
        print('無資料:'+str(dealer_id)+str(dealer_name)+str(sub_dealer_id)+str(sub_dealer_name))
    time.sleep(0.01)
    #break;

資料get:(牛牛牛)亞證卷6010(牛牛牛)亞證卷6010
資料get:(牛牛牛)亞證卷6010(牛牛牛)亞-網路6012
資料get:(牛牛牛)亞證卷6010(牛牛牛)亞-鑫豐0036003000310064
資料get:口袋證卷6620口袋證卷6620
資料get:土　　銀1030土　　銀1030
資料get:土　　銀1030土銀台中1031
資料get:土　　銀1030土銀台南1032
資料get:土　　銀1030土銀高雄1033
資料get:土　　銀1030土銀嘉義1034
資料get:土　　銀1030土銀新竹1035
資料get:土　　銀1030土銀玉里1036
資料get:土　　銀1030土銀花蓮1037
資料get:土　　銀1030土銀和平1038
資料get:土　　銀1030土銀士林1039
資料get:土　　銀1030土銀建國0031003000330041
資料get:土　　銀1030土銀彰化0031003000330042
資料get:土　　銀1030土銀白河0031003000330043
資料get:土　　銀1030土銀南港0031003000330046
資料get:大和國泰8890大和國泰8890
資料get:大    昌6460大    昌6460
資料get:大    昌6460大昌樹林6461
資料get:大    昌6460大昌安康6462
資料get:大    昌6460大昌新竹6463
資料get:大    昌6460大昌新店6464
資料get:大    昌6460大昌桃園6465
資料get:大    展5050大    展5050
資料get:大    展5050大展台南5058
資料get: 中國信託      6160 中國信託      6160
資料get: 中國信託      6160 中信三重      6161
資料get: 中國信託      6160 中信託忠      6162
資料get: 中國信託      6160 中信託永      6163
資料get: 中國信託      6160 中信託文      6164
資料get: 中國信託      6160 中信託新      6165
資料get: 中國信託      6160 中信松江      6167
資料get: 中國信託   

In [154]:
import pandas as pd
import os
import glob

def merge_broker_data(folder_path):
    """
    合併指定資料夾下所有券商資料的 CSV 檔案
    
    Parameters:
    folder_path (str): CSV 檔案所在的資料夾路徑
    
    Returns:
    pandas.DataFrame: 合併後的資料
    """
    # 取得資料夾中所有 CSV 檔案
    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    
    # 存放所有檔案的資料
    all_data = []
    
    # 讀取每個 CSV 檔案
    for file in csv_files:
        try:
            # 使用 big5 編碼讀取檔案
            df = pd.read_csv(file, encoding='utf-8')
            
            # 確保資料欄位名稱一致
            required_columns = ['券商名稱','個股名稱', '買進張數', '賣出張數', '買賣超張數']
            if not all(col in df.columns for col in required_columns):
                print(f"警告: 檔案 {file} 缺少必要欄位，已跳過")
                continue
                
            # 填補可能的空值為 0
            df = df.fillna(0)
            
            # 確保數值欄位為數值型態
            for col in ['買進張數', '賣出張數', '買賣超張數']:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
            
            all_data.append(df)
            
        except Exception as e:
            print(f"處理檔案 {file} 時發生錯誤: {str(e)}")
            continue
    
    if not all_data:
        raise ValueError("沒有成功讀取任何檔案")
    
    # 合併所有資料
    merged_data = pd.concat(all_data, ignore_index=True)
    
    return merged_data,all_data

def save_merged_data(merged_data, output_path):
    """
    儲存合併後的資料
    
    Parameters:
    merged_data (pandas.DataFrame): 合併後的資料
    output_path (str): 輸出檔案路徑
    """
    merged_data.to_csv(output_path, encoding='utf-8', index=False)

In [156]:
start_date = "2025-1-22"
folder_path=f'/Users/tai-huachiang/Downloads/卷商分點資料/{start_date}'
output_path=f'/Users/tai-huachiang/Downloads/卷商分點資料/{start_date}_all.csv'
mmerged_data,all_data=merge_broker_data(folder_path)
save_merged_data(mmerged_data, output_path)

處理檔案 /Users/tai-huachiang/Downloads/卷商分點資料/2025-1-22/broker_data_張數_豐銀_5500_豐銀_5500_2025-1-22.csv 時發生錯誤: No columns to parse from file
處理檔案 /Users/tai-huachiang/Downloads/卷商分點資料/2025-1-22/broker_data_張數_豐興_5690_豐興_5690_2025-1-22.csv 時發生錯誤: No columns to parse from file
處理檔案 /Users/tai-huachiang/Downloads/卷商分點資料/2025-1-22/broker_data_張數_凱基    _9200_凱基天母_0039003200310046_2025-1-22.csv 時發生錯誤: No columns to parse from file
處理檔案 /Users/tai-huachiang/Downloads/卷商分點資料/2025-1-22/broker_data_張數_富順_7530_富順_7530_2025-1-22.csv 時發生錯誤: No columns to parse from file
處理檔案 /Users/tai-huachiang/Downloads/卷商分點資料/2025-1-22/broker_data_張數_合　　庫_1020_合庫國際 _0031003000320051_2025-1-22.csv 時發生錯誤: No columns to parse from file
處理檔案 /Users/tai-huachiang/Downloads/卷商分點資料/2025-1-22/broker_data_張數_國票綜合_7790_國票國際_0037003700390062_2025-1-22.csv 時發生錯誤: No columns to parse from file
處理檔案 /Users/tai-huachiang/Downloads/卷商分點資料/2025-1-22/broker_data_張數_永豐金  _9A00_永豐國際證卷_0039004100380038_2025-1-22.csv 時發生錯誤: No columns to p

In [148]:
all_data

[     券商名稱             個股名稱  買進張數  賣出張數  買賣超張數
 0    元大華山               台塑   213     3    210
 1    元大華山               聯電   118     1    117
 2    元大華山  00637L元大滬深300正2    58     0     58
 3    元大華山               強茂    40     0     40
 4    元大華山    00968B元大優息投等債    31     0     31
 ..    ...              ...   ...   ...    ...
 95   元大華山            環宇-KY    21    25     -4
 96   元大華山               友達     0     4     -4
 97   元大華山            貿聯-KY     3     7     -4
 98   元大華山              華星光     8    11     -3
 99   元大華山               昆盈     0     3     -3
 
 [100 rows x 5 columns],
      券商名稱            個股名稱  買進張數  賣出張數  買賣超張數
 0    元富草屯              聯電   586    11    575
 1    元富草屯    00641R富邦日本反1   200     0    200
 2    元富草屯             力積電   113    14     99
 3    元富草屯           昇陽半導體   123    60     63
 4    元富草屯  00664R國泰臺灣加權反1    50     0     50
 ..    ...             ...   ...   ...    ...
 95   元富草屯              廣達     0     4     -4
 96   元富草屯             金山電     3     7   

In [150]:
mmerged_data

,券商名稱,個股名稱,買進張數,賣出張數,買賣超張數
0,元大華山,台塑,213,3,210
1,元大華山,聯電,118,1,117
2,元大華山,00637L元大滬深300正2,58,0,58
3,元大華山,強茂,40,0,40
4,元大華山,00968B元大優息投等債,31,0,31
...,...,...,...,...,...
80991,凱基桃園,00830國泰費城半導體,5,23,-18
80992,凱基桃園,友達,25,43,-18
80993,凱基桃園,玉山金,0,18,-18
80994,凱基桃園,台達電,1,18,-17
